# Basic Reduction of a Single Observation Using the SMURF pol2map Command

This tutorial demonstrates how to run a basic, standard reduction of a SCUBA-2 POL-2 dataset using the Starlink software suite. The basic command we will use is [`pol2map`](http://starlink.eao.hawaii.edu/cgi-bin/htxserver/sun258.htx/sun258.html?xref_POL2MAP) in [`SMURF`](http://starlink.eao.hawaii.edu/docs/sun258.htx/sun258.html). We will have to run this command twice: first to generate the Q and U timestreams, as well as an initial unpolarized intensity map. Secondly . we run the command again with a different invocation, to generate better masks and to produce the output Q, U and I maps as well as the final vector catalogue.

Although the results produced may improve with subsequent Starlink releases, the command invocation is not expected to change significantly.

For further background and details on the material covered in this tutorial, please consult the [`POL-2 Cookbook`](http://starlink.eao.hawaii.edu/docs/sc22.htx/sc22.html) which is also included in your Starlink software installation.

Before proceeding further you should ensure the Starlink software is loaded in your terminal, and you have loaded package `smurf`. You will need to replace the path in `export STARLINK_DIR` below with the location of your Starlink installation.

We'll also remove any previous pol2map settings that exist in your default `ADAM_USER` directory (`~/adam/`).

In [1]:
export STARLINK_DIR=/stardev
source $STARLINK_DIR/etc/profile
smurf



        SMURF commands are now available -- (Version 1.6.1)

        Type smurfhelp for help on SMURF commands.
        Type 'showme sun258' to browse the hypertext documentation.
        Type 'showme sc20' to view the Heterodyne DR cookbook.
        Type 'showme sc21' to view the SCUBA-2 DR cookbook.
        Type 'showme sc22' to view the POL-2 DR cookbook.

    


We're going to make a working directory and copy over the tutorial data tarball. We'll then untar the datafiles. You'll need to replace the path with the location you have downloaded the tutorial data to.

In [2]:
mkdir POL-2
cd POL-2
cp /ftp/pub/jcmt/usersmeetings/JCMT_POL-2_tutorial1_2017_raw_only.tar.gz .
tar xfz JCMT_POL-2_tutorial1_2017_raw_only.tar.gz

The dataset used in this example is made up of just one observation. Observation 43 is a POL-2 4.4′ × 4.4′ scan using scan mode `POLCV_DAISY`, and had a duration of 931s. The raw files are in the directory `tutorial/raw`, and there is an empty directory `tutorial/reduced` to be used for the output data.

First of all we need to create a list of the input file names, with the absolute paths included, and put it in the output directory we will be using. 

In [3]:
cd tutorial/raw
ls -d -1 $PWD/*.sdf > ../reduced/inputfiles.lis

We then change into the output directory.

In [4]:
cd ../reduced

By default `pol2map` will use a system temp directory to put the temperorary files. However, there can be issues (particularly on Macs) with this path length being too long. Therefore we recommend that you specifically set the directory for the temporary files produced under the pol2map. Do this using:

In [5]:
mkdir tmp
export STAR_TEMP=tmp/

## Initial run of `pol2map`: Q & U timestreams

The pol2map command may then be run on the raw data we have. Our first run of the command will a) run [`calcqu`](http://starlink.eao.hawaii.edu/cgi-bin/htxserver/sun258.htx/sun258.html?xref_CALCQU) to generate the Q&U timestreams from the raw POL-2/SCUBA-2 observation, and then b) create our initial unpolarized intensity (I) map. 

In [6]:
pol2map in=^inputfiles.lis iout=iauto qout=! uout=! \
mapdir=maps qudir=qudata

Logging to file pol2map.log 
Calculating Q, U and I time streams from raw analysed intensity data...
   1/1: Processing 116 raw data files from observation 20160125_00043 ... 

Making I map from 20160125_00043_0003...



The `inputfiles.lis` file created above is used to specify the list of files that will be used for input. The `iout` parameter is set to specify that an unpolarized total intensity map file called `iauto.sdf` will be produced, using an automatically-generated astronomical mask. Note that `qout` and `uout` are set to null values (the `!` in the above command), as no Q or U maps are to be produced during this initial reduction stage. Finally, local target directories called `maps/` and `qudata`/ are specified as the locations where the output maps and Q and U timestreams  respectively  will be written. Lots of details of this process are written into the logfile, `pol2map.log1`.

You can examine this first output I map (`iauto.sdf`) if you wish, by opening in e.g. Gaia.

## Second run of `pol2map`: the final output maps.
We can now produce the final I, Q and U maps, as well as the vector catalogs.

Now that an initial I map is available for use in IP (Instrumental Polarization) correction, the last stages of the polarimetry reduction can be run together via a single command. The following invocation of pol2map will generate the final intensity and polarimetric maps, and an associated vector catalogue

In [7]:
pol2map in=qudata/\* iout=iext qout=qext uout=uext mapdir=maps \
mask=iauto maskout1=astmask maskout2=pcamask ipref=iext cat=mycat.FIT \
debias=yes binsize=12

Logging to file pol2map.log 
(existing file pol2map.log moved to pol2map.log.1)

Masking will be based on SNR values derived from 'iauto'.

Making I map from 20160125_00043_0003...

The output vector catalogue will be based on maps that are binned up to 12.0 arcsec pixels.

Making Q map from 20160125_00043_0003...


Making U map from 20160125_00043_0003...

Creating the output catalogue: 'mycat.FIT'...


   5047 vectors written to the output catalogue.




The parameters specified in the above command are as follows:

 - `in`: The input Q, U and I time-stream data files to be used. These were created by the first run of pol2map described above.
 
 - `iout`: The resultant final total intensity map, named `iext.sdf` in this example.
 - `qout`: The resultant combined final Q map; named `qext.sdf` in this example.
 - `uout`: The resultant combined final U map; named `uext.sdf` in this example.
 - `mapdir`: The directory containing the individual I, Q and U maps from each separate observation` _Imap.sdf`, `_Qmap.sdf` and `_Umap.sdf`.
 - `mask`: The preexisting mask to be used as input during the reduction, named `iauto.sdf` in this example. This was created in step 4 (above).
 - `maskout1`: The first (AST) output mask generated by this pol2map run and used in the creation of the final I, Q and U maps; named `astmask.sdf` in this example.
 - `maskout2`: The second (PCA) output mask generated by this pol2map run and used in the creation of the final I, Q and U maps; named `pcamask.sdf` in this example.
 - `ipref`: The non-polarimetric reference map to be used for IP correction; named `iext` in this example. The specified file (`iext.sdf`) is the same one specified for parameter `iout` – the script first creates it as an output and then uses it as an input.
 - `cat`: The output FITS file generated containing the polarization vector catalogue; named `mycat.FIT` in this example.
 - `debias`: This parameter is set to `TRUE` here to specify that a correction for statistical bias is to be made to the percentage polarization and polarized intensity values in the output vector catalogue (see above).
 - `binsize`: Set this parameter to the bin size you wish to use for your output vector catalogue. Changing this value is a good first check of reliability.